In [25]:
from __future__ import print_function

import keras.backend.tensorflow_backend as K

from keras.models import Model
from keras.layers import Input, LSTM, Dense, Embedding, Flatten
import numpy as np


In [2]:
# from google.colab import drive
# drive.mount('/content/gdrive')

In [3]:
# Path to the data txt file on disk.
# data_path = '/content/gdrive/My Drive/ai_data/kor-eng/kor.txt'
input_data_path = '/data1/translation_data/korean-english-park.train/korean-english-park.train.en'
output_data_path = '/data1/translation_data/korean-english-park.train/korean-english-park.train.ko'

In [4]:
class Dataset:
    def __init__(self, data_path_list):
        self.data_path_list = data_path_list
    
    def _read_data_seq2seq(self, data_path_list):
        input_texts = []
        target_texts = []
        input_characters = set()
        target_characters = set()
        
        with open(data_path_list[0], 'r', encoding='utf-8') as f:
            input_lines = f.read().split('\n')
        
        with open(data_path_list[1], 'r', encoding='utf-8') as f:
            output_lines = f.read().split('\n')
            
        print('Number of lines:', len(input_lines), len(output_lines))
        for input_text, target_text in zip(input_lines, output_lines):
        #     input_text, target_text = line.split('\t')
            # We use "tab" as the "start sequence" character
            # for the targets, and "\n" as "end sequence" character.
            target_text = '\t' + target_text + '\n'
            input_texts.append(input_text)
            target_texts.append(target_text)
            for char in input_text:
                if char not in input_characters:
                    input_characters.add(char)
            for char in target_text:
                if char not in target_characters:
                    target_characters.add(char)
        
        input_characters = sorted(list(input_characters))
        target_characters = sorted(list(target_characters))
        num_encoder_tokens = len(input_characters)
        num_decoder_tokens = len(target_characters)
        max_encoder_seq_length = max([len(txt) for txt in input_texts])
        max_decoder_seq_length = max([len(txt) for txt in target_texts])

        print('Number of samples:', len(input_texts))
        print('Number of unique input tokens:', num_encoder_tokens)
        print('Number of unique output tokens:', num_decoder_tokens)
        print('Max sequence length for inputs:', max_encoder_seq_length)
        print('Max sequence length for outputs:', max_decoder_seq_length)

        input_token_index = dict(
            [(char, i) for i, char in enumerate(input_characters)])
        target_token_index = dict(
            [(char, i) for i, char in enumerate(target_characters)])
        
        encoder_input_data = np.zeros(
            shape=(len(input_texts), max_encoder_seq_length, 1),
            dtype='float32')
        decoder_input_data = np.zeros(
            (len(input_texts), max_decoder_seq_length, 1),
            dtype='float32')
        decoder_target_data = np.zeros(
            (len(input_texts), max_decoder_seq_length, 1),
            dtype='float32')

        for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
            for t, char in enumerate(input_text):
                encoder_input_data[i, t, 0] = input_token_index[char]
            for t, char in enumerate(target_text):
                # decoder_target_data is ahead of decoder_input_data by one timestep
                decoder_input_data[i, t, 0] = target_token_index[char]
                if t > 0:
                    # decoder_target_data will be ahead by one timestep
                    # and will not include the start character.
                    decoder_target_data[i, t - 1, 0] = target_token_index[char]
        
        # Always dataset comes first then additional info comes next
        return [[encoder_input_data, decoder_input_data, decoder_target_data], [num_encoder_tokens, num_decoder_tokens]]
    
    def get_data(self, data_type='translation'):
        data_type_dict = {
            'translation': self._read_data_seq2seq
        }
        all_data = None
        try:
            all_data = data_type_dict[data_type](data_path_list=self.data_path_list)
        except Exception as e:
            print('not valid data type. given:', data_type)
            print(e)
        
        return all_data
        

In [6]:
dataset = Dataset(data_path_list=[input_data_path, output_data_path])
args = dataset.get_data(data_type='translation')

Number of lines: 94124 94124
Number of samples: 94124
Number of unique input tokens: 142
Number of unique output tokens: 2206
Max sequence length for inputs: 605
Max sequence length for outputs: 379


In [36]:
class Sequence2sequence:
    def __init__(self, initial_params, gpu=0):
        
        max_encoder_seq_length, max_decoder_seq_length, num_encoder_tokens, num_decoder_tokens = initial_params
        
        latent_dim = 256  # Latent dimensionality of the encoding space.
        
        with K.tf.device('/gpu:' + str(gpu)):
            # Define an input sequence and process it.
            encoder_inputs = Input(shape=(max_encoder_seq_length, 1))
            encoder_inputs_flat = Flatten()(encoder_inputs)
            encoder_embed = Embedding(num_encoder_tokens, latent_dim, input_length=max_encoder_seq_length)(encoder_inputs_flat)
            encoder = LSTM(latent_dim, return_state=True)
            encoder_outputs, state_h, state_c = encoder(encoder_embed)
            # We discard `encoder_outputs` and only keep the states.
            encoder_states = [state_h, state_c]

            # Set up the decoder, using `encoder_states` as initial state.
            decoder_inputs = Input(shape=(max_decoder_seq_length, 1))
            decoder_inputs_flat = Flatten()(decoder_inputs)
            decoder_embed = Embedding(num_decoder_tokens, latent_dim, input_length=max_decoder_seq_length)(decoder_inputs_flat)
            # We set up our decoder to return full output sequences,
            # and to return internal states as well. We don't use the
            # return states in the training model, but we will use them in inference.
            decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
            decoder_outputs, _, _ = decoder_lstm(decoder_embed,
                                                 initial_state=encoder_states)
            # decoder_dense = Dense(num_decoder_tokens, activation='softmax')
            decoder_dense = Dense(1)
            decoder_outputs = decoder_dense(decoder_outputs)

            # Define the model that will turn
            # `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
        model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

        # Run training
        model.compile(optimizer='rmsprop', loss='mse')
        print(model.summary())
        
        self.model = model
    
    def train(self, epochs=50, batch_size=64, verbose=1, validation_split=0.1, *args):
        print(len(args))
        encoder_input_data, decoder_input_data, decoder_target_data = args[0]
        self.model.fit([encoder_input_data, decoder_input_data], 
                       decoder_target_data,
                       batch_size=batch_size, 
                       epochs=epochs,
                       verbose=verbose,
                       validation_split=validation_split)
        # Save model
        model.save('s2s.h5')
    
    @staticmethod
    def get_initial_params(*args):
#         print(len(args))
#         print(len(args[0]))
        (encoder_input_data, decoder_input_data, decoder_target_data), (num_encoder_tokens, num_decoder_tokens) = args[0]
        
        return [encoder_input_data.shape[1], decoder_input_data.shape[1], num_encoder_tokens, num_decoder_tokens]

In [45]:
seq2seq = Sequence2sequence(Sequence2sequence.get_initial_params(args), gpu=1)
seq2seq.train(50, 64, 1, 0.1, args[0])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_21 (InputLayer)           (None, 605, 1)       0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           (None, 379, 1)       0                                            
__________________________________________________________________________________________________
flatten_19 (Flatten)            (None, 605)          0           input_21[0][0]                   
__________________________________________________________________________________________________
flatten_20 (Flatten)            (None, 379)          0           input_22[0][0]                   
__________________________________________________________________________________________________
embedding_

KeyboardInterrupt: 

In [ ]:
# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())


def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)